In [ ]:
import warnings

from core.backtesting import BacktestingEngine

warnings.filterwarnings("ignore")


backtesting = BacktestingEngine(load_cached_data=True)

### STRATEGY PARAMETERS

In [ ]:
from hummingbot.strategy_v2.executors.position_executor.data_types import TrailingStop
from app.controllers.directional_trading.ema_crossover import EMACrossoverControllerConfig
import datetime
from decimal import Decimal
import pandas as pd
import plotly.express as px
connector_name = "binance"
trading_pair = "ZEC-USDT"
candles_connector = "binance"
interval = "1m"

# EMA crossover params
fast_ema = 9
slow_ema = 21


total_amount_quote = 1000
take_profit = 0.01  # 1%
stop_loss = 0.005   # 0.5%
trailing_stop_activation_price = 0.004
trailing_stop_trailing_delta = 0.002
max_executors_per_side = 2
time_limit = 60 * 60 * 12   # 12 hours
cooldown_time = 60 * 10     # 10 minutes

# timeframe for backtest
start = int(datetime.datetime(2025, 10, 20).timestamp())
end = int(datetime.datetime(2025, 10, 25).timestamp())

config = EMACrossoverControllerConfig(
    connector_name=connector_name,
    trading_pair=trading_pair,
    candles_connector=candles_connector,
    candles_trading_pair=trading_pair,
    interval=interval,
    fast_ema=fast_ema,
    slow_ema=slow_ema,
    total_amount_quote=Decimal(total_amount_quote),
    take_profit=Decimal(take_profit),
    stop_loss=Decimal(stop_loss),
    trailing_stop=TrailingStop(
        activation_price=Decimal(trailing_stop_activation_price),
        trailing_delta=Decimal(trailing_stop_trailing_delta)
    ),
    time_limit=time_limit,
    max_executors_per_side=max_executors_per_side,
    cooldown_time=cooldown_time,
)




###  RUN BACKTEST

In [ ]:
backtesting_result = await backtesting.run_backtesting(config, start, end, interval)

In [ ]:
print(backtesting_result.get_results_summary())
backtesting_result.get_backtesting_figure()

### EXECUTORS DATAFRAME

In [ ]:
executors_df = backtesting_result.executors_df
executors_df.head()


### PNL SCATTER PLOT

In [ ]:
executors_df['profitable'] = executors_df['net_pnl_quote'] > 0

fig = px.scatter(
    executors_df,
    x="timestamp",
    y="net_pnl_quote",
    title="PNL per Trade (EMA Crossover)",
    color="profitable",
    color_discrete_map={True: 'green', False: 'red'},
    labels={"timestamp": "Timestamp", "net_pnl_quote": "Net PNL (Quote)"},
    hover_data=["filled_amount_quote", "side"]
)

fig.update_layout(
    xaxis_title="Timestamp",
    yaxis_title="Net PNL (Quote)",
    legend_title="Profitable",
    font=dict(size=12, color="white"),
    showlegend=False,
    plot_bgcolor='rgba(0,0,0,0.8)',
    paper_bgcolor='rgba(0,0,0,0.8)',
    xaxis=dict(gridcolor="gray"),
    yaxis=dict(gridcolor="gray")
)

fig.add_hline(y=0, line_dash="dash", line_color="lightgray")
fig.show()


### Histogram of PNL Distribution

In [ ]:
fig = px.histogram(
    executors_df,
    x='net_pnl_quote',
    title='PNL Distribution (EMA Crossover)'
)
fig.show()